Using firmware from
    https://micropython.org/download/ESP32_GENERIC/


In [3]:
%serialconnect

serial exception on close write failed: [Errno 5] Input/output error
Found serial ports: /dev/ttyUSB1, /dev/ttyS0 
Connecting to --port=/dev/ttyUSB1 --baud=115200 
[(b'DoESLiverpool', b'\xfa\x9f\xc2\xf4.\xf4', 1, -46, 3, False), (b'DoESTest', b'\xfe\x9f\xc2\xf4.\xf4', 1, -47, 5, False), (b'Find Out Digital', b'\xf0\x9f\xc2\xf4\x1a\x7f', 6, -51, 3, False), (b'DoESLiverpool', b'\xfa\x9f\xc2\xf4\x1a\x7f', 6, -51, 3, False), (b'DoESTest', b'\xfe\x9f\xc2\xf4\x1a\x7f', 6, -52, 5, False), (b'', b'\xf6\x9f\xc2\xf4\x1a\x7f', 6, -52, 3, False), (b'', b'\x02\x9f\xc2\xf4\x1a\x7f', 6, -56, 3, False), (b'Top Floor Wifi', b'\xfe\xec\xda\x07\x08k', 11, -57, 3, False), (b'', b'\x0e\xec\xda\x07\x08k', 11, -57, 3, False), (b'spscontroller', b'\xd8:\xdd\x8d\x9e0', 1, -64, 3, False), (b'DIRECT-E3-HP DeskJet 2600 series', b'\x9c{\xef\xcd\xe5\xe4', 6, -73, 3, False), (b'', b'*\xe8)\xe4\xdc\xa8', 6, -73, 3, False), (b'DoESLiverpool', b'\xfa\x9f\xc2\xf4\x1b\xe4', 1, -75, 3, False), (b'DoESTest', b'\xfe\x9f\xc2

In [256]:
%sendtofile --source /home/julian/repositories/micropython-mqtt/mqtt_as/mqtt_as.py

Sent 800 lines (29331 bytes) to mqtt_as.py.


In [242]:
%sendtofile config.txt

wifiname     DoESLiverpool
wifipassword decafbad00
boardname    esp32arcade
mqttbroker   mosquitto.doesliverpool.xyz

Sent 4 lines (116 bytes) to config.txt.


In [ ]:
%sendtofile config.txt

wifiname     spscontroller
wifipassword puppet-24
boardname    esp32arcade
mqttbroker   192.168.4.1

In [27]:
%serialconnect

serial exception on close write failed: [Errno 5] Input/output error
Found serial ports: /dev/ttyUSB1, /dev/ttyS0 
Connecting to --port=/dev/ttyUSB1 --baud=115200 
[(b'DoESTest', b'\xfe\x9f\xc2\xf4.\xf4', 1, -56, 5, False), (b'DoESLiverpool', b'\xfa\x9f\xc2\xf4.\xf4', 1, -57, 3, False), (b'Find Out Digital', b'\xf0\x9f\xc2\xf4\x1a\x7f', 6, -57, 3, False), (b'', b'\x02\x9f\xc2\xf4\x1a\x7f', 6, -57, 3, False), (b'', b'\xf6\x9f\xc2\xf4\x1a\x7f', 6, -57, 3, False), (b'DoESLiverpool', b'\xfa\x9f\xc2\xf4\x1a\x7f', 6, -58, 3, False), (b'DoESTest', b'\xfe\x9f\xc2\xf4\x1a\x7f', 6, -59, 5, False), (b'Top Floor Wifi', b'\xfe\xec\xda\x07\x08k', 11, -62, 3, False), (b'', b'\x0e\xec\xda\x07\x08k', 11, -62, 3, False), (b'Faraday', b'\x18\xe8)\xe4\xdc\xa8', 6, -69, 3, False), (b'', b'\x1a\xe8)\xe4\xdc\xa8', 6, -69, 3, False), (b'', b'*\xe8)\xe4\xdc\xa8', 6, -69, 3, False), (b'DoESLiverpool', b'\xfa\x9f\xc2\xf4\x1b\xe4', 1, -72, 3, False), (b'DoESTest', b'\xfe\x9f\xc2\xf4\x1b\xe4', 1, -74, 5, False), (

In [266]:
%sendtofile --source stdmqttas.py
%sendtofile --quiet --source utils.py

Sent 88 lines (2992 bytes) to stdmqttas.py.
Sent 39 lines (1151 bytes) to utils.py.


In [4]:
%sendtofile main.py

import network
w = network.WLAN()
w.active(1)
w.disconnect()
print(w.scan())

from mqtt_as import MQTTClient
from stdmqttas import fconfig, config, mqttconnecttask, callbackcmdtask, flashledconnectedtask
import uasyncio as asyncio
from stdmqttas import pinled, flashpinled, shortmac, topicstem, itertools_count
import time

from arcadeboards1 import setupallarcadeboards, setbuttonlight, newbuttondown
from arcadeboards1 import row1, row2, brow1, brow2, bigredbutton, bigredlight

topicmessage = topicstem+"/message"
topiccmd = topicstem+"/cmd"
topicstatus = topicstem+"/status"
topicreply = topicstem+"/reply"
topiconline = topicstem+"/online"
topicip = topicstem+"/ip"

topicrow1 = "spscontroller/controller/video"
topicrow2 = "spscontroller/controller/audio"

row1mess = [ "Videos/purple.mp4", "Videos/white.mp4", "Videos/red.mp4", "Videos/yellow.mp4", "Videos/blue.mp4", "Videos/green.mp4" ]
row2mess = [ "Music/silent.mp3", "Music/nostalgia.mp3", "Music/joyful.mp3", "Music/silly.mp3", "Music/romantic.mp3", "Music/tense.mp3" ] 

dimlight = 500
brightlight = 50000
async def arcadetask(client):
    setupallarcadeboards()
    while not client.isconnected():
        setbuttonlight(row1[0], 50000)
        await asyncio.sleep_ms(200)
        setbuttonlight(row1[0], 5000)
        await asyncio.sleep_ms(600)

    i1, i2 = 0, 0
    currentrow1button = row1[i1]
    currentrow2button = row2[i2]
    await client.publish(topicrow1, row1mess[i1])
    await client.publish(topicrow2, row2mess[i2])
    redlightflashes = 0
    while True:
        k = newbuttondown(50)
        if k in brow1:
            i1 = brow1.index(k)
            if currentrow1button:
                setbuttonlight(currentrow1button, dimlight)
            currentrow1button = row1[i1]
            setbuttonlight(currentrow1button, brightlight)
        if k in brow2:
            i2 = brow2.index(k)
            if currentrow2button:
                setbuttonlight(currentrow2button, dimlight)
            currentrow2button = row2[i2]
            setbuttonlight(currentrow2button, brightlight)
        if k == bigredbutton:
            setbuttonlight(bigredlight, brightlight)
            await client.publish(topicrow1, row1mess[i1])
            await client.publish(topicrow2, row2mess[i2])
            redlightflashes = 20
        await asyncio.sleep_ms(50)
        if redlightflashes > 0:
            redlightflashes -= 1
            setbuttonlight(bigredlight, brightlight if (redlightflashes%2) else 0)
            
        
def callbackcmd(topic, msg, retained):
    print("callbackcmd", topic, msg)
    if topic == topiccmd:
        aloop.create_task(callbackcmdtask(client, topicreply, msg))

async def onconnecttask(client):
    ipnumber = client._sta_if.ifconfig()[0]
    await client.publish(topicstatus, ipnumber, retain=True)
    await client.subscribe(topiccmd)
    await client.subscribe(topicmessage)
    await client.publish(topicip, ipnumber, retain=True, qos=1)
    client.connectioncount += 1
    print("subscribed")
            
config['subs_cb'] = callbackcmd
config['connect_coro'] = onconnecttask
config['server'] = fconfig["mqttbroker"]
config['ssid'] = fconfig["wifiname"]
config['wifi_pw'] = fconfig["wifipassword"]

print(config)
client = MQTTClient(config)
client.DEBUG = True
client.connectioncount = 0

aloop = asyncio.get_event_loop()
aloop.create_task(arcadetask(client))
aloop.create_task(mqttconnecttask(client, None))
aloop.run_forever()


Sent 98 lines (3446 bytes) to main.py.


In [14]:
%serialconnect

serial exception on close write failed: [Errno 5] Input/output error
Found serial ports: /dev/ttyUSB1, /dev/ttyS0 
Connecting to --port=/dev/ttyUSB1 --baud=115200 
Traceback (most recent call last):
  File "main.py", line 1, in <module>
  File "mqtt_as.py", line 37, in <module>
KeyboardInterrupt: 
MicroPython v1.22.2 on 2024-02-22; Generic ESP32 module with ESP32
Type "help()" for more information.
>>> 
>>> 
raw REPL; CTRL-B to exit
>Ready.


In [15]:
%sendtofile arcadeboards1.py

import machine, struct, time

addrs = [ 0x3a, 0x3c, 0x3e ]
row1 = sum([(addr,13), (addr,1)] for addr in addrs, [])
row2 = sum([(addr,12), (addr,0)] for addr in addrs, [])

brow1 = sum([(addr,19), (addr,2)] for addr in addrs, [])
brow2 = sum([(addr,18), (addr,20)] for addr in addrs, [])

bigredbutton = (0x3b, 18)
bigredlight = (0x3b, 12)

buttonsdown = { }

i2c = None
button_pins = (18, 19, 20, 2)
def setupallarcadeboards():
    global i2c
    i2c = machine.I2C(1) # , sda=machine.Pin(26), scl=machine.Pin(25))
    print(i2c)
    aaddrs = i2c.scan()
    print("i2c devices found", list(map(hex, aaddrs)))
    assert set(aaddrs).issuperset([0x3a, 0x3b, 0x3c, 0x3e])
    k = sum((1 << pin)  for pin in button_pins)
    for addr in [0x3a, 0x3b, 0x3c, 0x3e]:
        i2c.writeto(addr, bytearray([0x00, 0x07, 0xFF])) # reset device
        i2c.writeto(addr, bytearray([0x00, 0x01]))
        print("chiptype ", addr, i2c.readfrom(addr, 1))  #_ATTINY817_HW_ID_CODE = const(0x87)
        for s in [0x03, 0x0b, 0x05]:        # set all buttons to PULL_UP
            i2c.writeto(addr, struct.pack(">bbI", 0x01, s, k))


def setbuttonlight(pp, v):
    i2c.writeto(pp[0], bytearray([0x08, 0x01, pp[1], (v >> 8), v & 0xFF]))

def newbuttondown(holdtimems):
    for addr in [0x3a, 0x3b, 0x3c, 0x3e]:
        i2c.writeto(addr, bytearray([0x01, 0x04]))
        time.sleep(0.008)
        vb = i2c.readfrom(addr, 4)
        v = struct.unpack(">I", vb)[0]
        tms = time.ticks_ms()
        for p in button_pins:
            k = (addr, p)
            if (v & (1 << p)):
                if k in buttonsdown:
                    del buttonsdown[k]
            else:
                if k in buttonsdown:
                    if buttonsdown[k] != 0 and tms - buttonsdown[k] > holdtimems:
                        buttonsdown[k] = 0
                        return k
                else:
                    buttonsdown[k] = tms
    return None
 

Sent 56 lines (1926 bytes) to arcadeboards1.py.


In [10]:
from arcadeboards1 import setupallarcadeboards, setbuttonlight, newbuttondown
setupallarcadeboards()
while 1:
    a = newbuttondown(10)
    if a:
        print(a)

I2C(1, scl=25, sda=26, freq=430107)
i2c devices found ['0x3a', '0x3b', '0x3c', '0x3e']
chiptype  58 b'\x87'
chiptype  59 b'\x87'
chiptype  60 b'\x87'
chiptype  62 b'\x87'
(58, 18)
(59, 18)
(59, 18)
(59, 18)
..........................

*** Sending Ctrl-C



Traceback (most recent call last):
  File "<stdin>", line 4, in <module>
  File "arcadeboards1.py", line 36, in newbuttondown
KeyboardInterrupt: 


In [11]:
print(hex(59))

0x3b
